<a href="https://colab.research.google.com/github/silver5938/NLP_Twitter_Sentiment/blob/main/NLP_Twitter_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**" 딥러닝을 이용한 트위터 감성 분석 project "**

1. 학습 데이터 : AI허브에서 가져온 감성 train 데이터를 사용
2. 테스트 데이터 : lucy api를 이용해 데이터 10,000건 추출

프로젝트 세팅

In [ ]:
# DATASET
DATASET_COLUMNS = ["seq", "age", "sex", "keywords","health", "sentiment1", "sentiment2","text"]
DATASET_ENCODING = "CP949"
TRAIN_SIZE = 40879

# TEXT CLENAING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+" #http주소 제거

# WORD2VEC 
W2V_SIZE = 100 #워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
W2V_WINDOW = 5 #컨텍스트 윈도우 크기
W2V_EPOCH = 3 #학습을 위한 프로세스 수
W2V_MIN_COUNT = 3 #단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)


# SENTIMENT
불안 = '0'
슬픔 = '1'
분노 = '2'
기쁨 = '3'
상처 = '4'
당황 = '5'


# EXPORT
SENTIMENT_MODEL = "LSTM 이용" #감성 예측 모델
WORD2VEC_MODEL = gensim.models.word2vec.Word2Vec(size=W2V_SIZE, 
                                            window=W2V_WINDOW, 
                                            min_count=W2V_MIN_COUNT, 
                                            workers=W2V_EPOCH)
TOKENIZER_MODEL = "Tokenizer() 이용" #토크나이징
ENCODER_MODEL = "LabelEncoder() 이용" #문자를 수치화

기타 설정(import)


In [3]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 40 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 6.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595740 sha256=8a1f62c9c2ffe94821ee9015ac14004cac8326d108f797640db8fa6fef8e0c9d
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 9.4 MB/s 
     |████████████████████████████████| 769 kB 6.3 MB/s 
     |████████████████████████████████| 3.0 MB 14.4 MB/s 
     |████████████████████████████████| 895 kB 37.1 MB/s 


In [4]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-3pcl0ha_
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-3pcl0ha_
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12770 sha256=2799b53dd6d6d6870d751f5297962b694315a6fb142aa56b55ffe1951853533c
  Stored in directory: /tmp/pip-ephem-wheel-cache-rtz43utj/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [5]:
#pytorch
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

#GPU 사용
device = torch.device("cuda:0")

#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

# DataFrame
import pandas as pd

# Matplot
import matplotlib as mpl  # 기본 설정 만지는 용도
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm  # 폰트 관련 용도
import seaborn as sns
import missingno as msno
from plotly import graph_objs as go
plt.style.use('seaborn')
sns.set(font_scale=2.5) # matplotlib 의 기본 scheme 말고 seaborn scheme 을 세팅하고, 일일이 graph 의 font size 를 지정할 필요 없이 seaborn 의 font_scale 을 사용하면 편함

%matplotlib inline

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer

# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM
from keras import utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Word2vec
import gensim

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Set log
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


※ 한글 깨짐 현상을 해결하는 방법 ※

In [4]:
# Google Colab matplotlib 한글 폰트 오류 문제 해결

# 1. 나눔 폰트 다운
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 2. 런타임 다시 시작



Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 2s (5,567 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 160837 files and d

In [6]:
# 3. matplotlib의 폰트를 Nanum 폰트로 지정
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 

파일 업로드 - drive연동했다면 할 필요 없음

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from google.colab import files
uploaded = files.upload()

파일 불러오기 :  
한글은 utf-8형식이므로 깨지기 때문에 encoding='CP949' 로 설정 해 주어야 한다.

In [6]:
# colab과 google Drive 연동을 위한 인증 및 네트워크 디렉토리 마운트
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
df_train = pd.read_csv('/content/drive/MyDrive/dataset/s_training.CSV', encoding='CP949')
df_test = pd.read_csv('/content/drive/MyDrive/dataset/s_test.CSV', encoding='CP949')

head 메소드를 이용해 데이터 확인

In [8]:
df_test.head(5)

,i_docid,i_sitename,i_url,i_title,i_sdate,sentiment1
0,TW1419668235989880833,Twitter,https://twitter.com/KaiS2com/status/1419668235...,＂걱정돼서 전화했어요＂ 우도주막 걱정에 깜짝 전화한 에이스 카이ㅜㅜ(서윗) | tv...,20210726,기쁨
1,TW1419661732599844866,Twitter,https://twitter.com/HuffPostKorea/status/14196...,"""돈에 지배당한 것"" 서장훈이 쓰리잡으로 1억 모은 25살 의뢰인에게 한 말은 일리...",20210726,기쁨
2,TW1419672897426231297,Twitter,https://twitter.com/threadreaderapp/status/141...,"@aqualemine Namaste, please find the unroll he...",20210726,기쁨
3,TW1419667990836957184,Twitter,https://twitter.com/Angry82pigeon/status/14196...,@dullchip0 너무 좋은..생각이예요 모두 궁금한걸~?!?(슴췹님 그림 하나라...,20210726,기쁨
4,TW1419672897426231297,Twitter,https://twitter.com/threadreaderapp/status/141...,"@aqualemine Namaste, please find the unroll he...",20210726,기쁨


In [9]:
df_train.head(5)

,seq,age,sex,keywords,health,sentiment1,sentiment2,text
0,2,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,회사에서 중요한 큰 프로젝트를 나 혼자 하게 되니 솔직히 두려워.
1,3,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,회사에 직장 상사가 무섭게 생겨서 친해지는 게 두려워.
2,4,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,이번에 처음으로 힘들게 들어간 직장인데 첫 직장이라서 그런지 너무 긴장돼.
3,5,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,직장에서 사람들과의 관계에서 안 좋아질까 걱정돼.
4,7,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,회의 시간에 나의 직급이 낮아서 그런지 의견을 무시하는 경향이 있어서 의견을 제시하...


데이터 갯수 확인

In [10]:
train_len = len(df_train)
test_len = len(df_test)

In [11]:
train_len

40879

In [12]:
test_len

303

info 메소드 이용해 기본적인 데이터 정보 확인

In [13]:
df_train.info()
df_train.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40879 entries, 0 to 40878
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   seq         40879 non-null  int64 
 1   age         40879 non-null  object
 2   sex         40879 non-null  object
 3   keywords    40879 non-null  object
 4   health      40879 non-null  object
 5   sentiment1  40879 non-null  object
 6   sentiment2  40879 non-null  object
 7   text        40879 non-null  object
dtypes: int64(1), object(7)
memory usage: 2.5+ MB


seq           0
age           0
sex           0
keywords      0
health        0
sentiment1    0
sentiment2    0
text          0
dtype: int64

In [14]:
df_test.info()
df_test.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   i_docid     303 non-null    object
 1   i_sitename  303 non-null    object
 2   i_url       303 non-null    object
 3   i_title     303 non-null    object
 4   i_sdate     303 non-null    int64 
 5   sentiment1  303 non-null    object
dtypes: int64(1), object(5)
memory usage: 14.3+ KB


i_docid       0
i_sitename    0
i_url         0
i_title       0
i_sdate       0
sentiment1    0
dtype: int64


데이터 null값 확인(1)

In [15]:
for col in df_train.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)

column:        seq	 Percent of NaN value: 0.00%
column:        age	 Percent of NaN value: 0.00%
column:        sex	 Percent of NaN value: 0.00%
column:   keywords	 Percent of NaN value: 0.00%
column:     health	 Percent of NaN value: 0.00%
column: sentiment1	 Percent of NaN value: 0.00%
column: sentiment2	 Percent of NaN value: 0.00%
column:       text	 Percent of NaN value: 0.00%


In [16]:
for col in df_test.columns:
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (df_test[col].isnull().sum() / df_test[col].shape[0]))
    print(msg)

column:    i_docid	 Percent of NaN value: 0.00%
column: i_sitename	 Percent of NaN value: 0.00%
column:      i_url	 Percent of NaN value: 0.00%
column:    i_title	 Percent of NaN value: 0.00%
column:    i_sdate	 Percent of NaN value: 0.00%
column: sentiment1	 Percent of NaN value: 0.00%


데이터 정제(Data Cleaning)_1

In [17]:
df_train.drop_duplicates(subset = ['text'], inplace=True) # text 열에서 중복인 내용이 있다면 중복 제거
df_train['text'] = df_train['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
df_train['text'] = df_train['text'].str.replace('^ +', "") # 공백은 empty 값으로 변경
df_train['text'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
df_train = df_train.dropna(how='any') # Null 값 제거
print('전처리 후 학습용 샘플의 개수 :',len(df_train))

2021-08-02 01:46:17,070 : INFO : NumExpr defaulting to 2 threads.


전처리 후 학습용 샘플의 개수 : 39415


In [18]:
df_test.drop_duplicates(subset = ['i_title'], inplace=True) # i_title 열에서 중복인 내용이 있다면 중복 제거
df_test['i_title'] = df_test['i_title'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
df_test['i_title'] = df_test['i_title'].str.replace('^ +', "") # 공백은 empty 값으로 변경
df_test['i_title'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
df_test = df_test.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(df_test))

전처리 후 테스트용 샘플의 개수 : 297


In [19]:
df_train.sample(n=10)

,seq,age,sex,keywords,health,sentiment1,sentiment2,text
3142,3951,청년,여성,"진로, 취업, 직장",해당없음,상처,질투하는,같은 일을 하는데 선배가 항상 나보다 실적이 좋으니까 힘이 빠져
11235,14081,노년,남성,"건강, 죽음",만성질환 유,불안,혼란스러운,요즘 손바닥이 붉어져서 생기가 돈다고 생각했었는데 알고 보니 간 경화 증상이어서 불안해
4589,5738,청년,남성,대인관계,해당없음,상처,고립된,생일인데 축하해주는 사람 하나 없네 나에게 진짜 친구는 없는 것 같다
16807,21164,청년,여성,대인관계,해당없음,기쁨,신이 난,오늘 친구가 승진했다는 소식을 들어서 너무 기뻐
14788,18612,중년,남성,"대인관계(부부, 자녀)",해당없음,기쁨,흥분,퇴근하면 몸과 마음이 지쳐서 가정일에 소극적이 돼 본의 아니게 자녀에게도 화를 내게 돼
12746,15999,중년,남성,건강,해당없음,분노,노여워하는,내가 너무 자주 병원을 가는 것 같아
145,188,청소년,남성,학교폭력/따돌림,해당없음,슬픔,우울한,이제 도움을 청할 친구 자체가 없어
27736,34899,중년,여성,건강,해당없음,슬픔,환멸을 느끼는,아등바등 살아봐도 사는 것이 나아지지 않아 환멸을 느껴
31953,40170,노년,여성,"건강, 죽음",만성질환 유,불안,회의적인,치매에 걸리기라도 하는 날엔 자식들이 날 요양원으로 보내버릴 것 같아 불안해
35577,44754,청소년,남성,가족관계,해당없음,기쁨,흥분,오늘 엄마에게 칭찬을 받았어


데이터 확인

In [20]:
filteredData = df_train['sentiment1'].isin(['기쁨','분노','불안','슬픔','상처','당황'])
filterdd = df_train[~filteredData]

In [21]:
filterdd

,seq,age,sex,keywords,health,sentiment1,sentiment2,text


In [22]:
temp = df_train.groupby('sentiment1').count()['seq'].reset_index().sort_values(by='seq',ascending=False)
temp.style.background_gradient(cmap='Purples')

,sentiment1,seq
3,불안,7107
2,분노,6671
5,슬픔,6659
0,기쁨,6455
4,상처,6421
1,당황,6102


In [23]:
fig = go.Figure(go.Funnelarea(
    text =temp.sentiment1,
    values = temp.seq,
    title = {"position": "top center", "text": "Funnel-Chart of Sentiment Distribution"}
    ))
fig.show()

감성(sentiment1) 라벨링

In [24]:
df_train.loc[(df_train['sentiment1'] == "불안"), 'sentiment1'] = 0  #불안 => 0
df_train.loc[(df_train['sentiment1'] == "슬픔"), 'sentiment1'] = 1  #슬픔 => 1
df_train.loc[(df_train['sentiment1'] == "분노"), 'sentiment1'] = 2  #분노 => 2
df_train.loc[(df_train['sentiment1'] == "기쁨"), 'sentiment1'] = 3  #기쁨 => 3
df_train.loc[(df_train['sentiment1'] == "상처"), 'sentiment1'] = 4  #상처 => 4
df_train.loc[(df_train['sentiment1'] == "당황"), 'sentiment1'] = 5  #당황 => 5

In [25]:
data_list = []
for q, label in zip(df_train['text'], df_train['sentiment1'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [26]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['회사에서 중요한 큰 프로젝트를 나 혼자 하게 되니 솔직히 두려워', '0']
['내가 육십 세부터 받을 수 있는 연금 금액이 실망스러워 ', '1']
['내가 업무 때문에 직장동료에게 전화를 했는데 글쎄 직장동료 애인이 나를 내연녀로 오해했대', '5']
['결혼한 지 오 년째 아이가 생기지 않아서 눈물이 나', '1']
['새로운 부서에 발령을 받았는데 명예퇴직 대상자만 모아둔 곳이라 우울해', '1']
['모든 가족의 빚을 나 혼자 갚고 있어서 고립된 느낌이야 아무도 나를 도와주지 않아', '4']
['나 매우 화가 나', '2']


데이터 정제(Data Cleaning)_2

In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

In [ ]:
#불용어 제거
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']


In [ ]:
X_train = []
for sentence in df_train['text']:
    temp_X = mecab.morphs(sentence) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

In [ ]:
X_train[:10]

sklearn을 이용한 Train & Test 데이터 나누기

In [27]:
#train & test 데이터로 나누기
                                                     
dataset_train, dataset_test = train_test_split(data_list, test_size=0.20, random_state=0)

In [28]:
print(len(dataset_train))
print(len(dataset_test))

31532
7883


KoBERT 입력 데이터로 만들기

데이터를 train data와 test data로 나누었다면 각 데이터가 KoBERT 모델의 입력으로 들어갈 수 있는 형태가 되도록 토큰화, 정수 인코딩, 패딩 등을 해주어야 한다. 예시 코드에 입력데이터의 형태가 되도록 해주는 클래스가 있어서 동일하게 코드를 작성해주었다.

In [29]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

하이퍼 파라미터들을 조정해준다. Batch size는 32, epochs는 8, learning rate는 5e-5로 설정

In [30]:
# Setting parameters
max_len = 200
batch_size = 32
warmup_ratio = 0.1
num_epochs = 6
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

토크나이징(tokenizing)

In [31]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [32]:
data_train[0]

(array([   2, 4626, 7828, 4628, 7086, 2047, 5156, 6896, 4598, 7342, 7880,
        1375, 3129, 5859, 3166, 6273, 6542, 7096, 6844,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

torch형식의 dataset 만들기

In [33]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

KoBERT 학습모델 만들기

In [34]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [35]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

KoBERT 모델 학습시키기

In [36]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

epoch 1 batch id 1 loss 1.82370126247406 train acc 0.125
epoch 1 batch id 201 loss 1.6790648698806763 train acc 0.19962686567164178
epoch 1 batch id 401 loss 1.3734962940216064 train acc 0.31717581047381543
epoch 1 batch id 601 loss 1.1863712072372437 train acc 0.3956426788685524
epoch 1 batch id 801 loss 1.4051579236984253 train acc 0.4394506866416979

epoch 1 train acc 0.46714418526031104



epoch 1 test acc 0.5949231689363268


epoch 2 batch id 1 loss 1.0622477531433105 train acc 0.59375
epoch 2 batch id 201 loss 1.0965394973754883 train acc 0.5984141791044776
epoch 2 batch id 401 loss 0.9311253428459167 train acc 0.6024002493765586
epoch 2 batch id 601 loss 0.875831127166748 train acc 0.6120528286189684
epoch 2 batch id 801 loss 1.0122100114822388 train acc 0.6187968164794008

epoch 2 train acc 0.6250739519945909



epoch 2 test acc 0.6188236105999264


epoch 3 batch id 1 loss 0.8986563682556152 train acc 0.65625
epoch 3 batch id 201 loss 0.8941256403923035 train acc 0.6540733830845771
epoch 3 batch id 401 loss 0.808211088180542 train acc 0.6586658354114713
epoch 3 batch id 601 loss 0.6785659790039062 train acc 0.6675852745424293
epoch 3 batch id 801 loss 0.9608256220817566 train acc 0.673689138576779

epoch 3 train acc 0.6796082657200812


...


epoch 4 test acc 0.6257706109679794


epoch 5 batch id 1 loss 0.532193124294281 train acc 0.8125
epoch 5 batch id 201 loss 0.42650091648101807 train acc 0.7694340796019901
epoch 5 batch id 401 loss 0.6560627818107605 train acc 0.779925187032419
epoch 5 batch id 601 loss 0.3504335284233093 train acc 0.7906613976705491
epoch 5 batch id 801 loss 0.6137253642082214 train acc 0.7967774656679151

epoch 5 train acc 0.799315415821501



epoch 5 test acc 0.6266677401545823


epoch 6 batch id 1 loss 0.4794579744338989 train acc 0.84375
epoch 6 batch id 201 loss 0.4058549404144287 train acc 0.8101679104477612
epoch 6 batch id 401 loss 0.5670661926269531 train acc 0.8179551122194514
epoch 6 batch id 601 loss 0.26591864228248596 train acc 0.8239912645590682
epoch 6 batch id 801 loss 0.41112253069877625 train acc 0.827247191011236

epoch 6 train acc 0.8276179006085193



epoch 6 test acc 0.6266677401545823


예측 모델(문장 테스트)


입력되는 문장을 KoBERT의 입력 형식으로 바꿔주는 코드를 작성

In [42]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

#input, output 산출 가능한 class
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("불안")
            elif np.argmax(logits) == 1:
                test_eval.append("슬픔")
            elif np.argmax(logits) == 2:
                test_eval.append("분노")
            elif np.argmax(logits) == 3:
                test_eval.append("기쁨")
            elif np.argmax(logits) == 4:
                test_eval.append("상처")
            elif np.argmax(logits) == 5:
                test_eval.append("당황")
            else :
                test_eval.append("중립")

        print(">> 입력한 내용의 감성은 " + test_eval[0] + " 입니다.")

using cached model


In [49]:
#질문 무한반복하기! F 입력시 종료
end = 1
while end == 1 :
    sentence = input("문장을 입력하세요 : ")
    if sentence == 'F' :
        break
    predict(sentence)
    print("\n")

문장을 입력하세요 : 걱정돼서 전화했어요＂ 우도주막 걱정에 깜짝 전화한 에이스 카이ㅜㅜ(서윗)
>> 입력한 내용의 감성은 분노 입니다.


문장을 입력하세요 : 걱정됩니다
>> 입력한 내용의 감성은 불안 입니다.


문장을 입력하세요 : 소스윗
>> 입력한 내용의 감성은 당황 입니다.


문장을 입력하세요 : ㅜㅜ
>> 입력한 내용의 감성은 상처 입니다.


문장을 입력하세요 : 깜짝
>> 입력한 내용의 감성은 당황 입니다.


문장을 입력하세요 : 깜짝 전화한 에이스 카이ㅜㅜ(서윗)
>> 입력한 내용의 감성은 슬픔 입니다.


문장을 입력하세요 : 고마워요♡
>> 입력한 내용의 감성은 기쁨 입니다.


문장을 입력하세요 : 매주 월요일은 재환이와 함께하니까 월요병 없는 싱-나는 싱요…
>> 입력한 내용의 감성은 상처 입니다.


문장을 입력하세요 : 유네스코, '한국의 갯벌' 세계문화유산목록에 등재
>> 입력한 내용의 감성은 기쁨 입니다.


문장을 입력하세요 : 강한자만이 살아남는 k-출근 레전드 
>> 입력한 내용의 감성은 기쁨 입니다.


문장을 입력하세요 : 이 더위에 쿠팡 물류센터 노동자들은 변변한 냉방기구도 없이 찌는 환경에서 일해야 한다.
>> 입력한 내용의 감성은 상처 입니다.


문장을 입력하세요 : 미국서 모래폭풍으로 고속도로 20중 추돌…최소 7명 사망
>> 입력한 내용의 감성은 슬픔 입니다.


문장을 입력하세요 : 조민 친구 SNS에 글 올려 “조민, 세미나 참석 맞다...조국 가족에 사죄" 뒤집힌 진술
>> 입력한 내용의 감성은 상처 입니다.


문장을 입력하세요 : 지금은 잃어버린 꿈
>> 입력한 내용의 감성은 슬픔 입니다.


문장을 입력하세요 : "복음 전파한다" 사찰에 방화…40대, 2심도 실형
>> 입력한 내용의 감성은 분노 입니다.


문장을 입력하세요 : F
